In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import io
import boto3

import s3fs
fs = s3fs.S3FileSystem(anon=False,key='####',secret='#######')

from dotenv import load_dotenv
load_dotenv(verbose=True)

def aws_session(region_name='us-east-1'):
    return boto3.session.Session(aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'), #looks for any .env file
                                aws_secret_access_key=os.getenv('AWS_ACCESS_KEY_SECRET'), #Has to be in same directory
                                region_name=region_name) #from above

def make_bucket(name, acl): 
    session = aws_session()
    s3_resource = session.resource('s3')
    return s3_resource.create_bucket(Bucket=name, ACL=acl)

def upload_file_to_bucket(bucket_name, file_path):
    session = aws_session()
    s3_resource = session.resource('s3')
    file_dir, file_name = os.path.split(file_path)

    bucket = s3_resource.Bucket(bucket_name)
    bucket.upload_file(
      Filename=file_path,
      Key=file_name,
      ExtraArgs={'ACL': 'public-read'}
    )

    s3_url = f"https://{bucket_name}.s3.amazonaws.com/{file_name}"
    return s3_url

def download_file_from_bucket(bucket_name, s3_key, dst_path):
    session = aws_session()
    s3_resource = session.resource('s3')
    bucket = s3_resource.Bucket(bucket_name)
    bucket.download_file(Key=s3_key, Filename=dst_path)

## download_file_from_bucket('music-demo-lyrics', 'lyrics_25k.csv', 'short_name.csv')
## with open('short_name.csv') as fo:
    ## print(fo.read())

Investigating why the 25k dataframe got so big.  I think the pd.merge with multiple years per artist created multiple repeats.

In [2]:
big_df = pd.read_csv('Junk/Main_25k.csv')
big_df

,original_csv,artist_name,song_name,link,lyrics,language,genre,year
0,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1995
1,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1997
2,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1999
3,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,2000
4,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,2005
...,...,...,...,...,...,...,...,...
512664,25K,Noah Cyrus,July,../lyrics/noahcyrus/july.html,\n\nI've been holding my breath\nI've been cou...,NaN,NaN,2020
512665,25K,Noah Cyrus,July,../lyrics/noahcyrus/july.html,\n\nI've been holding my breath\nI've been cou...,NaN,NaN,Not Defined
512666,25K,Noah Cyrus,Stay Together,../lyrics/noahcyrus/staytogether.html,\n\nI drank straight to my head\nI went outsid...,NaN,NaN,2018
512667,25K,Noah Cyrus,Stay Together,../lyrics/noahcyrus/staytogether.html,\n\nI drank straight to my head\nI went outsid...,NaN,NaN,2020


In [3]:
big_df.describe(include='all')

,original_csv,artist_name,song_name,link,lyrics,language,genre,year
count,512669,512669,512669,512669,512669,0.0,0.0,512669
unique,1,141,19610,23141,23014,NaN,NaN,72
top,25K,Johnny Cash,Can't Help Falling In Love,../lyrics/elvispresley/hounddog.html,\n\nYou ain't nothin' but a hound dog\nCryin' ...,NaN,NaN,Not Defined
freq,512669,52845,523,441,441,NaN,NaN,23795
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
big_df[big_df.duplicated(subset=['artist_name','song_name'])]

,original_csv,artist_name,song_name,link,lyrics,language,genre,year
1,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1997
2,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1999
3,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,2000
4,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,2005
5,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,2007
...,...,...,...,...,...,...,...,...
512662,25K,Noah Cyrus,I Got So High That I Saw Jesus,../lyrics/noahcyrus/igotsohighthatisawjesus.html,"\n\nI got so high that I saw Jesus\nHe said, i...",NaN,NaN,Not Defined
512664,25K,Noah Cyrus,July,../lyrics/noahcyrus/july.html,\n\nI've been holding my breath\nI've been cou...,NaN,NaN,2020
512665,25K,Noah Cyrus,July,../lyrics/noahcyrus/july.html,\n\nI've been holding my breath\nI've been cou...,NaN,NaN,Not Defined
512667,25K,Noah Cyrus,Stay Together,../lyrics/noahcyrus/staytogether.html,\n\nI drank straight to my head\nI went outsid...,NaN,NaN,2020


In [6]:
big_df[big_df.duplicated]

KeyError: Index([       ('25K', ''N Sync ', 'All I Want Is You This Christmas', '../lyrics/nsync/alliwantisyouthischristmas.html', '\n\nSnowy Night\nWhere it's warm here by the fire\nHere with you (here with you)\nI have all that I desire\nIt's been a long year\nBut somehow we got by\nNow it's Christmas Eve\nAnd love is on our side\nI don't need a hundred gifts beneath the tree\nDon't you know the best thing you could give to me\nNothin' else will do,\nAll I want is you this Christmas\n\nChildren sing (children sing)\nChristmas carols at our door\nSpreading joy, (spreading joy)\nThat's what Christmas time is for\nThough we don't have much\nYou've got me and I've got you\nAnd with a little faith\nMiracles come true\n\nAnd I don't need a hundred gifts beneath the tree\nDon't you know the best thing you could give to me\nNothin' else will do,\nAll I want is you this Christmas\n\nI don't want the love any other girl \nI want only thing in this whole wide world\nNothing else will do\nAll I 

In [5]:
big_df.shape

(512669, 8)

In [7]:
big_df.drop_duplicates(subset=['artist_name','song_name'], inplace = True)
big_df.describe(include='all')

,original_csv,artist_name,song_name,link,lyrics,language,genre,year
count,22947,22947,22947,22947,22947,0.0,0.0,22947
unique,1,141,19610,22947,22860,NaN,NaN,51
top,25K,Johnny Cash,Intro,../lyrics/philcollins/forafriend.html,\n\n[Instrumental]\n,NaN,NaN,2005
freq,22947,781,15,1,9,NaN,NaN,1356
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
big_df.shape

(22947, 8)

In [10]:
big_df.to_csv('main_25k_df.csv', index = False)
upload_file_to_bucket('music-lyrics','main_25k_df.csv')

'https://music-lyrics.s3.amazonaws.com/main_25k_df.csv'

In [15]:
!aws s3 ls 'worm-begin'

2021-04-07 08:38:33     105621 album_details_25k.csv
2021-04-07 08:09:22   27655251 decades_tcc_ceds_music.csv
2021-04-07 08:34:22     348120 genres_artists_data.csv
2021-04-07 08:19:38  276163416 genres_lyrics_data.csv
2021-04-07 08:22:31  187407184 labeled_lyrics_cleaned.csv
2021-04-07 08:38:18   40754845 lyrics_25k.csv
2021-04-07 08:38:34    2276377 songs_details_25k.csv


In [32]:
last_df = pd.read_csv('s3://worm-begin/labeled_lyrics_cleaned.csv')
last_df.describe(include='all')

,Unnamed: 0,artist,seq,song,label
count,158353.000000,158353,158353,158353,158353.000000
unique,NaN,14691,135991,99031,NaN
top,NaN,Elvis Presley,"Somewhere over the rainbow, way up high\r\nThe...",Have Yourself a Merry Little Christmas,NaN
freq,NaN,821,167,162,NaN
mean,79176.000000,NaN,NaN,NaN,0.491052
std,45712.717926,NaN,NaN,NaN,0.249619
min,0.000000,NaN,NaN,NaN,0.000000
25%,39588.000000,NaN,NaN,NaN,0.286000
50%,79176.000000,NaN,NaN,NaN,0.483000
75%,118764.000000,NaN,NaN,NaN,0.691000


In [33]:
last_df.head(20)

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.6260
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.6300
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.2400
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.5360
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.3710
5,5,Elijah Blake,I just want to ready your mind\r\n'Cause I'll ...,Uno,0.3210
6,6,Elijah Harris,To believe\r\nOr not to believe\r\nThat is the...,Girlfriend (Main),0.6010
7,7,Elis,Dieses ist lange her.\r\nDa ich deine schmalen...,Abendlied,0.3330
8,8,Elis,A child is born\r\nOut of the womb of a mother...,Child,0.5060
9,9,Elis,Out of the darkness you came \r\nYou looked so...,Come to Me,0.1790


In [34]:
last_df[last_df.duplicated(subset=['artist','song'])]

,Unnamed: 0,artist,seq,song,label
9970,9970,Andy Williams,"Honey babe, oh, honey babe\r\nMmm, honey babe,...",I Like Your Kind of Love,0.7670
9971,9971,Andy Williams,Where's this place called Lonely Street?\r\n\r...,Lonely Street,0.3890
9976,9976,Andy Williams,"(So rare, so rare)\r\n(So rare, so rare)\r\n\r...",So Rare,0.6830
42862,42862,Failure,I heard you driving in my car\r\nThen in a fro...,Stuck on You,0.2770
43183,43183,T.I.,Look at this little n**** man...\r\nHey T.I. i...,Da Dopeman,0.5540
45717,45717,Thunder,Well my temperature's rising and my feet are o...,Gimme Some Lovin',0.3550
45718,45718,Thunder,"So tired of waiting, I walked an empty land\r\...",Love Walked In,0.2810
45719,45719,Thunder,I should've seen it coming now\r\nShould've kn...,River of Pain,0.4970
50648,50648,Carl Perkins,"Ain't no doubt about it, this must be love\r\n...",Glad All Over,0.8680
57111,57111,Kiss,Alright\nI remember the day that we met\nI nee...,I Stole Your Love,0.4900


In [35]:
last_df[last_df.duplicated(subset=['artist','song'])].count()

Unnamed: 0    41
artist        41
seq           41
song          41
label         41
dtype: int64

In [36]:
last_df.drop_duplicates(subset=['artist','song'], inplace = True)
last_df[last_df.duplicated(subset=['artist','song'])].count()

Unnamed: 0    0
artist        0
seq           0
song          0
label         0
dtype: int64

In [37]:
last_df.head(5)

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371


In [38]:
last2_df = last_df.rename(columns={'artist':'artist_name','seq':'lyrics','song':'song_name'})
last2_df.head(5)

,Unnamed: 0,artist_name,lyrics,song_name,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371


In [39]:
last3_df = pd.DataFrame(last2_df, columns=['original_csv','artist_name','song_name','link','lyrics','language','genre','date'])
last3_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
0,NaN,Elijah Blake,Everyday,NaN,"No, no\r\nI ain't ever trapped out the bando\r...",NaN,NaN,NaN
1,NaN,Elijah Blake,Live Till We Die,NaN,"The drinks go down and smoke goes up, I feel m...",NaN,NaN,NaN
2,NaN,Elijah Blake,The Otherside,NaN,She don't live on planet Earth no more\r\nShe ...,NaN,NaN,NaN
3,NaN,Elijah Blake,Pinot,NaN,"Trippin' off that Grigio, mobbin', lights low\...",NaN,NaN,NaN
4,NaN,Elijah Blake,Shadows & Diamonds,NaN,"I see a midnight panther, so gallant and so br...",NaN,NaN,NaN


In [42]:
last3_df.describe(include='all')

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
count,0.0,158312,158312,0.0,158312,0.0,0.0,0.0
unique,NaN,14691,99031,NaN,135966,NaN,NaN,NaN
top,NaN,Elvis Presley,Have Yourself a Merry Little Christmas,NaN,"Somewhere over the rainbow, way up high\r\nThe...",NaN,NaN,NaN
freq,NaN,821,162,NaN,167,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
last3_df[last3_df.duplicated(subset=['artist_name','song_name'])]

,original_csv,artist_name,song_name,link,lyrics,language,genre,date


In [45]:
last3_df[last3_df.duplicated(subset=['lyrics'])]

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
205,NaN,Ella Fitzgerald,"Gee Baby, Ain't I Good to You",NaN,Love makes me treat you the way that I do\nGee...,NaN,NaN,NaN
289,NaN,Ella Fitzgerald,"Oh, Lawd, I'm On My Way",NaN,Porgy and all\r\nI'm on my way to a heav'nly l...,NaN,NaN,NaN
339,NaN,Ella Fitzgerald,But Not for Me [Mono Mix][*],NaN,"They're writing songs of love, but not for me\...",NaN,NaN,NaN
369,NaN,Ella Fitzgerald,The Christmas Song [Alternate Take][#][*],NaN,Chestnuts roasting on an open fire\r\nJack Fro...,NaN,NaN,NaN
500,NaN,Ella Fitzgerald,More Than You Know [Take 1] [#][*][Take],NaN,"Whether you are here or yonder, \r\nWhether yo...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
158314,NaN,Miriam Klein,Body and Soul,NaN,"My heart is sad and lonely\r\nFor you I sigh, ...",NaN,NaN,NaN
158315,NaN,Miriam Klein,I Cried for You,NaN,I remember other days how I used to weep \r\nO...,NaN,NaN,NaN
158318,NaN,Miriam Makeba,Brand New Day,NaN,When all the dark clouds roll away\r\nAnd the ...,NaN,NaN,NaN
158320,NaN,Miriam Makeba,For What It's Worth,NaN,There's something happening here \r\nWhat it i...,NaN,NaN,NaN


that is a lot of 'same' lyrics.  Droping those...

In [46]:
last3_df.drop_duplicates(subset=['lyrics'], inplace = True)

In [47]:
last3_df[last3_df.duplicated(subset=['lyrics'])]


,original_csv,artist_name,song_name,link,lyrics,language,genre,date


In [48]:
last3_df.describe(include='all')

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
count,0.0,135966,135966,0.0,135966,0.0,0.0,0.0
unique,NaN,10776,94925,NaN,135966,NaN,NaN,NaN
top,NaN,Elvis Presley,Intro,NaN,Wakin' up in a twin XL\r\nI got drunk in a che...,NaN,NaN,NaN
freq,NaN,746,132,NaN,1,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
last3_df['original_csv'] = 'big_no_genre'

In [52]:
last3_df.describe(include='all')

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,original
count,135966,135966,135966,0.0,135966,0.0,0.0,0.0,135966
unique,1,10776,94925,NaN,135966,NaN,NaN,NaN,1
top,big_no_genre,Elvis Presley,Intro,NaN,Wakin' up in a twin XL\r\nI got drunk in a che...,NaN,NaN,NaN,big_no_genre
freq,135966,746,132,NaN,1,NaN,NaN,NaN,135966
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
last3_df.drop(columns = ['original'], inplace=True)
last3_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
0,big_no_genre,Elijah Blake,Everyday,NaN,"No, no\r\nI ain't ever trapped out the bando\r...",NaN,NaN,NaN
1,big_no_genre,Elijah Blake,Live Till We Die,NaN,"The drinks go down and smoke goes up, I feel m...",NaN,NaN,NaN
2,big_no_genre,Elijah Blake,The Otherside,NaN,She don't live on planet Earth no more\r\nShe ...,NaN,NaN,NaN
3,big_no_genre,Elijah Blake,Pinot,NaN,"Trippin' off that Grigio, mobbin', lights low\...",NaN,NaN,NaN
4,big_no_genre,Elijah Blake,Shadows & Diamonds,NaN,"I see a midnight panther, so gallant and so br...",NaN,NaN,NaN


In [57]:
last3_df.keys()

Index(['original_csv', 'artist_name', 'song_name', 'link', 'lyrics',
       'language', 'genre', 'date', 'original'],
      dtype='object')

In [59]:
last3_df.to_csv('big_no_genre_df.csv', index = False)
upload_file_to_bucket('music-lyrics','big_no_genre_df.csv')

'https://music-lyrics.s3.amazonaws.com/big_no_genre_df.csv'